In [7]:
from __future__ import print_function
import pyzbar.pyzbar as pyzbar
import numpy as np
import cv2
import yaml
import math
# 0.17m
length_ofedge=0.17
RUN_ON_CAR=True
from collections import defaultdict


In [8]:
%matplotlib notebook
import matplotlib.pyplot as plt

from picar import front_wheels, back_wheels
from picar.SunFounder_PCA9685 import Servo
import picar
from time import sleep
import cv2
import cv2.cv as cv
import numpy as np
import picar
from math import *
picar.setup()
import numpy as np

bw = back_wheels.Back_Wheels()
fw = front_wheels.Front_Wheels()
pan_servo = Servo.Servo(1)
tilt_servo = Servo.Servo(2)
picar.setup()
# - left
# + right
# to be measured directly on the picar
offset = 9
# pan_servo.offset = 10
# tilt_servo.offset = 0
initial_angle_fw=90+offset
bw.speed = 0
fw.turn(initial_angle_fw)
pan_servo.write(90)
tilt_servo.write(100)

# motor_speed = 70
# 70i*3=127cm, i= 1.27/210 m/s
# 70i=1.27/3=0.42333333333333334m/s
# 40i~100i=0.2419047619~0.60476190476 m/s
L=0.14
true_speed_unit=0.15/35
MAX_SPEED=100*true_speed_unit
MIN_SPEED=70*true_speed_unit
MIN_RADIAN=1.0/180*pi
MAX_TRUN_LEFT=(initial_angle_fw-120)*pi/180
MAX_TRUN_RIGHT=(initial_angle_fw-60)*pi/180

DEBUG "back_wheels.py": Set debug off
DEBUG "TB6612.py": Set debug off
DEBUG "TB6612.py": Set debug off
DEBUG "PCA9685.py": Set debug off
DEBUG "front_wheels.py": Set debug off
DEBUG "front_wheels.py": Set wheel debug off
DEBUG "Servo.py": Set debug off


In [ ]:
objp = np.array([[0., 0., 0.],
       [0., length_ofedge, 0.],
       [length_ofedge, length_ofedge, 0.],
       [length_ofedge, 0., 0.]])
with open('calibration.yaml') as f:
    loadeddict = yaml.load(f)

mtx = np.array(loadeddict.get('camera_matrix'))
dist = np.array(loadeddict.get('dist_coeff'))
axis = np.float32([[0.1,0,0], [0,0.1,0], [0,0,-0.1]]).reshape(-1,3)
waypoints=[(0.,0.,0.),(1.,0.,0.),(2.,2.,pi),(0.,0.,0.)]
land_mark_pos={'Landmark 1':(2.0,0.,-pi/2),'Landmark 2':(2.,2.5,0),'GOAL':(0.,0.,-pi)}

In [9]:
def decode(im) : 
    # Find barcodes and QR codes
    
    decodedObjects = pyzbar.decode(im)

    # Print results
    for obj in decodedObjects:
        print('Type : ', obj.type)
        print('Data : ', obj.data,'\n')

    return decodedObjects
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img
# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6
 
 
# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :
 
    assert(isRotationMatrix(R))
     
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])





In [10]:
def controller(current_position_pose,waypoint,L,precision,v_min,v_max,k_rho=1,k_alpha=1*5,k_beta=-1*2):
    """controller that takes the current position and desired position, then yields a control signal for v and gamma.
    Automatically round to the max/min speed if exceeding the limits. This is unstable."""
    (x,y,theta)=current_position_pose
    (xg,yg,thetag)=waypoint
    delta_x=xg-x
    delta_y=yg-y
    rho=sqrt(delta_x**2+delta_y**2)
    if rho<=precision:
        return 0,0
    alpha=atan2(delta_y,delta_x)-theta
    beta=-theta-alpha+thetag
    if -pi/2<alpha<=pi/2:
        direction=1
    else:
        direction=-1
        alpha_prime=-pi-beta
        beta_prime=-pi-alpha
        alpha=alpha_prime
        beta=beta_prime
    # Control signal
    v=k_rho*rho*direction
    if v==0:
        return 0,0
    if abs(v)<v_min:
        v=v_min*direction
    if abs(v)>v_max:
        v=v_max*direction
    omega=(k_alpha*alpha+k_beta*beta)
    gamma=atan(omega*L/abs(v))
    print (v*direction,omega*direction,gamma)
    gamma=np.clip(gamma,MAX_TRUN_LEFT ,MAX_TRUN_RIGHT)
    return v*direction,gamma
def controller_cs(current_position_pose,waypoint,L,precision,v_min,k_rho=1.0,k_alpha=1*5,k_beta=-1*2):
    """controller that takes the current position and desired position, then yields a control signal for v and gamma.
    Keep using constant speed v_min as the speed output. It is stable."""
    (x,y,theta)=current_position_pose
    (xg,yg,thetag)=waypoint
    delta_x=xg-x
    delta_y=yg-y
    rho=sqrt(delta_x**2+delta_y**2)
    if rho<=precision:
        return 0,0
    alpha=atan2(delta_y,delta_x)-theta
    beta=-theta-alpha+thetag
    if -pi/2<alpha<=pi/2:
        direction=1
    else:
        direction=-1
        alpha_prime=-pi-beta
        beta_prime=-pi-alpha
        alpha=alpha_prime
        beta=beta_prime
    # Control signal
    k=v_min/rho
    ratio=k/k_rho
    k_alpha=k_alpha*ratio
    k_beta=k_beta*ratio
    v=v_min*direction
    if v==0:
        return 0,0
    omega=(k_alpha*alpha+k_beta*beta)
    gamma=atan(omega*L/abs(v))
#     gamma=round(abs(gamma))
    gamma=np.clip(gamma,MAX_TRUN_LEFT ,MAX_TRUN_RIGHT)
#     print (v*direction,omega*direction,gamma)
    return v*direction,gamma
def isclose(a, b, rel_tol=5e-01, abs_tol=0.1):
    """Approximatiy check for two coordinates pair. Determine if the two coordinates are close enough"""
    x,y,_=a
    xg,yg,_=b
    deltad=sqrt((xg-x)**2+(yg-y)**2)
    return abs(deltad-0) <= max(rel_tol * max(abs(deltad), abs(0)), abs_tol)

In [12]:

def display(im, decodedObjects):
    """Display barcode and QR code location and calculate the position and orientation of the camera, also calculates the 
    rotation matrix according to QR codes"""
    dist=np.array([])
    pos=defaultdict(list)
    angle=defaultdict(list)
    # Loop over all decoded objects
    for decodedObject in decodedObjects: 
        if decodedObject.type != 'QRCODE':
            continue
        
        land_mark=decodedObject.data
        points = decodedObject.location
#         print (points)
        points=[point for point in points]
        # If the points do not form a quad, find convex hull
        if len(points) > 4 : 
            hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
            hull = list(map(tuple, np.squeeze(hull)))
        else : 
            hull = points;

        # Number of points in the convex hull
        n = len(hull)
        
        
        if any(h[0]>=2147483647 or h[1]>=2147483647 for h in hull):
            continue
        if not RUN_ON_CAR:
            # Draw the convext hull
            for j in range(0,n):
                if hull[j][0]>=2147483647 or hull[j][1]>=2147483647 or hull[(j+1) % n][0]>=2147483647 or hull[(j+1) % n][1]>=2147483647:
                    continue
                cv2.line(im, hull[j], hull[ (j+1) % n], (255,0,0), 3)
            font = cv2.FONT_HERSHEY_SIMPLEX
            for h in hull:
    #             if h[0]>=2147483647 or h[1]>=2147483647:
    #                 continue
                cv2.putText(im,str(h),h, font, 1,(255,255,255),2,cv2.LINE_AA)
        
        
        ret,rvecs, tvecs = cv2.solvePnP(objp, np.array(hull).astype(np.float32), mtx, dist)
        Rt = np.zeros(shape=(3,3))
        cv2.Rodrigues(rvecs,Rt)
        R = Rt.transpose()
        euler_angles=rotationMatrixToEulerAngles(R)
        temp=-R.dot(tvecs)
        temp[2]=temp[2]+0.4
        pos[land_mark].append(temp)
        angle[land_mark].append([euler_angle*180/math.pi for euler_angle in euler_angles][1])
        if not RUN_ON_CAR:
            cv2.putText(im,'Position of camera: '+str(pos),(50,50), font, 0.5,(255,255,255),2,cv2.LINE_AA)
            cv2.putText(im,'Euler angles of object: '+str([euler_angle*180/math.pi for euler_angle in euler_angles][1]),(50,100), font, 0.5,(255,255,255),2,cv2.LINE_AA)
#         print (pos)
    
        
#         print (imgpts)
        if not RUN_ON_CAR:
                # project 3D points to image plane
            imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
            im = draw(im,np.array(hull).astype(np.float32),imgpts)
        
        print (pos)
        print (angle)
        if not RUN_ON_CAR:
            cv2.imshow("Results", im);
            cv2.waitKey(1);
    # Display results 
    return pos,angle
     

In [14]:
def pose_estimator(land_mark_pos,cap,current_position_pose,true_speed,theta,deltat,direction,gamma,mtx,dist,number_of_frame=20):
#     trying to localize with landmark
    """localizing the car according to QR codes. Automatically combines the result of multiple QR codes"""
    
    poses_dict=defaultdict(list)
    angles_dict=defaultdict(list)
    for _ in range(number_of_frame):
        _, frame = cap.read()
        frame = cv2.undistort(frame, mtx, dist)
        
        decodedObjects = decode(frame)
        
        pose,angle=display(frame, decodedObjects)
#         print(pose,angle)
        if len(pose.values())>0:
            for landmark_name,p in pose.items():
                poses_dict[landmark_name]+=p
            for landmark_name,a in angle.items():
                angles_dict[landmark_name]+=a
    if len(poses_dict.values())>0:
        current_position_pose=[]
        print (angles_dict)
        for landmark_name,p in poses_dict.items():
            land_mark=land_mark_pos[landmark_name]
            poses=np.array(p)
            angles=np.array(angles_dict[landmark_name])
            pose=np.average(poses,axis=0)
            print (angles)
            angle=np.average(angles)
            dis=np.linalg.norm(pose)
            print ('averaged: '+str((pose,angle))+'dis: '+str(dis))
            if (abs(angle)<=10):
                angle=0
            angle=angle*pi/180
            
            temp=np.array((land_mark[0]-dis*cos(angle),land_mark[1]+dis*sin(angle),pi/2-angle+land_mark[2]))
            print (temp)
            current_position_pose.append(temp)
            print (current_position_pose)
        current_position_pose=np.array(current_position_pose)
        current_position_pose=np.average(current_position_pose,axis=0)
#         print (current_position_pose)
        current_position_pose=tuple(current_position_pose)
        print (current_position_pose)
    else:
        delta_position_pose=(true_speed*cos(theta)*deltat*direction,true_speed*sin(theta)*deltat*direction,true_speed/L*tan(gamma)*deltat*direction)
#         print delta_position_pose
        current_position_pose=tuple(map(sum, zip(current_position_pose, delta_position_pose)))
    
    
    

    return current_position_pose

In [16]:

# debug_mode=False
# deltat=0.5
# current_position_pose=initial_position_pose
# current_angle_fw=initial_angle_fw
# archived_x_y_theta=[]
# cap = cv2.VideoCapture(0)
def automatic_driver(waypoints,current_position_pose,deltat,land_mark_pos):
    """Driver that automatically drive the car to the way points and calibrates the current position and orientation based on
    the QR codes"""
    initial_position_pose=waypoints[0]
    fw.turn(initial_angle_fw)
    for waypoint in waypoints: 
        print ('heading to ' +str(waypoint))
        reached=False
        while not reached:

            x,y,theta=current_position_pose
            #         v,gamma=controller(current_position_pose,waypoint,L,deltat*MIN_SPEED,MIN_SPEED,MAX_SPEED)
            v,gamma=controller_cs(current_position_pose,waypoint,L,deltat*MIN_SPEED,MIN_SPEED)
            if v<0:
                direction=-1
            else:
                direction=1
            v=abs(v)
            if not debug_mode:
                fw.turn(current_angle_fw-gamma*180/pi)
                sleep(1)
            print ('turning angle: '+str(current_angle_fw-gamma*180/pi))
            motor_speed=int(round(abs(v)/true_speed_unit))
            if not debug_mode:
                bw.speed = motor_speed
            true_speed=motor_speed*true_speed_unit
            if direction==1:
                bw.backward()
            elif direction==-1:
                bw.forward()
            if not debug_mode:
                sleep(deltat)
            bw.stop()
    #         fw.turn(initial_angle_fw)
            current_position_pose=pose_estimator(land_mark_pos,cap,current_position_pose,true_speed,theta,deltat,direction,gamma,mtx,dist)
            archived_x_y_theta.append(current_position_pose)
            if(isclose(current_position_pose,waypoint,abs_tol=deltat*MIN_SPEED) or v==0):
                    reached=True
                    print (str(waypoint)+' reached')
                    fw.turn(current_angle_fw)
        if not debug_mode:
                raw_input("Press Enter to continue...")
    

In [32]:
def foward_controller(t,speed=70):
    sleep(2)
    bw.speed=speed
    bw.backward()
    sleep(t)
    bw.stop()
def counter_clock_controller(t=1.7,speed=70,angle=60):
    bw.stop()
    fw.turn(angle)
    sleep(2)
    bw.speed=speed
    bw.backward()
    sleep(t)
    bw.stop()
def pose_and_angle(cap,mtx, dist):
    _, frame = cap.read()
    frame = cv2.undistort(frame, mtx, dist)
    decodedObjects = decode(frame)
    pose,angle=display(frame, decodedObjects)
    return pose,angle

In [82]:
for _ in range(2):
    fw.turn(initial_angle_fw)
    pose,_=pose_and_angle(cap,mtx, dist)
    if len(pose.values())>10:
        dis=pose.values()[0][0][2]
        while dis>0.5:
            foward_controller(0.5)
            pose,_=pose_and_angle(cap,mtx, dist)
            dis=pose[2]
    else:
        foward_controller(3.5)
    raw_input("Press Enter to continue...")

counter_clock_controller()

for _ in range(2):
    fw.turn(initial_angle_fw)
    pose,_=pose_and_angle(cap,mtx, dist)
    if len(pose.values())>0:
        dis=pose.values()[0][0][2]
        while dis>0.5:
            foward_controller(0.5)
            pose,_=pose_and_angle(cap,mtx, dist)
            dis=pose[2]
    else:
        foward_controller(1.5)
counter_clock_controller()
raw_input("Press Enter to continue...")
counter_clock_controller(1.2)
for _ in range(5):
    fw.turn(initial_angle_fw)
    pose,_=pose_and_angle(cap,mtx, dist)
    if len(pose.values())>0:
        dis=pose.values()[0][0][2]
        while dis>0.5:
            foward_controller(0.5)
            pose,_=pose_and_angle(cap,mtx, dist)
            dis=pose[2]
    else:
        foward_controller(1.5)
counter_clock_controller(3)

Press Enter to continue...s
Press Enter to continue...s
Press Enter to continue...j


KeyboardInterrupt: 